<a href="https://colab.research.google.com/github/leotuni/Nicolas-stuff/blob/master/Corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Corona data cause I'm bored 
I am importing all the data from the GITHUB REPOSITORY CSSEGISandData, which is run by John Hopkins University. 

In [0]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

In [0]:
#@title Import and Option to show print more data 
import pandas as pd 
import glob
#plotting 
import plotly.express as px


pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', frame.shape[0]+1)

I import all the files and concatenate them into one dataframe (**frame**) 

I then: 

0.   they used Mainlad china and China, so replaced mainland china to china 
1.   change **Last Update** to Date format, and sort by Date
2.   Fill the NaN of **State/Province** with the **Country/Region** values to have a full collumn
3.   Subsett by **Last Update** and **State/Province** and drop all Duplicates

The reason there is Duplicates is that many coutries didn't report new data every day, so a file from date: 1/2/2020 would have the data from Sweden of the 31/1/2020 if There is no new **Last Update**



In [0]:
#@title Import files and Prepare data 

path = r'/content/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports' # use your path
all_files = glob.glob(path + "/*.csv") #collect all files in one
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['file']=filename[-14:]
    li.append(df)
#Make one file and sort
frame = pd.concat(li, axis=0, ignore_index=True, sort=False) #one dataframe 
frame['Country/Region'] = frame['Country/Region'].str.replace('Mainland China','China')
frame['Last Update'] = pd.to_datetime(frame['Last Update'])
frame = frame.sort_values(by="Last Update")
frame['Province/State'].fillna(frame['Country/Region'], inplace=True) #putting the Coutry in teh NaN of Province for data cleanup 

frame = frame.assign(day=frame['Last Update'].dt.date).\
          drop_duplicates(subset=(['day', 'Province/State'])).drop('day',axis=1)
frame['Deaths'].fillna(0, inplace=True)
frame['Recovered'].fillna(0, inplace=True)

frame.head()


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,file
1316,Sichuan,China,2020-01-22 17:00:00,5.0,0.0,0.0,NaN,NaN,01-22-2020.csv
1318,Tianjin,China,2020-01-22 17:00:00,4.0,0.0,0.0,NaN,NaN,01-22-2020.csv
1317,Taiwan,Taiwan,2020-01-22 17:00:00,1.0,0.0,0.0,NaN,NaN,01-22-2020.csv
1315,Shanxi,China,2020-01-22 17:00:00,1.0,0.0,0.0,NaN,NaN,01-22-2020.csv
1314,Shanghai,China,2020-01-22 17:00:00,9.0,0.0,0.0,NaN,NaN,01-22-2020.csv


In [0]:
#@title Data analysis
frame['Total_deaths_by_country'] = frame.groupby([frame['Last Update'].dt.date ,frame['Country/Region']])['Deaths'].transform('sum')
frame['Total_Cases_by_country'] = frame.groupby([frame['Last Update'].dt.date ,frame['Country/Region']])['Confirmed'].transform('sum')
frame['Death Percentage'] = frame['Total_deaths_by_country']/ frame['Total_Cases_by_country'] *100
frame['Death Percentage']=frame['Death Percentage'].round(2)
filter_Europe= frame['Country/Region'].isin(['Ukraine','Italy', 'Germany', 'France', 'Spain', 'Denmark', 'Netherlands','North Ireland','Malta','Iceland','Slovakia','Lithuania','Hungary','Switzerland','Norway','Greece', 'North Macedonia', 'Romania', 'Czech Republic','Austria','UK','Sweden','Belgium','Portugal','Poland', 'Republic of Ireland', 'Finland', 'Croatia', 'Slovenia'])
# 'Ukraine' was excluded as it fucks up the stats (too few cases still )
# frame['Country/Region'].unique()

#Plots

In [0]:
#@title Total deaths by country 
fig = px.line(frame, x='Last Update', y='Total_deaths_by_country', color='Country/Region', title=' Total deaths by country')
fig.show()

In [0]:
#@title Total cases by country 
fig = px.line(frame, x='Last Update', y='Total_Cases_by_country', color='Country/Region', title=' Total cases by country')
fig.show()

## Death Percentage vs Number of Confirmed cases 

*   Italy has a very high death rate compared with the otehr countries, perhaps beccuase it is further
*   Germany stands out as a particularly low death rate


From this plot we see that Death % incrases as the you go furthre into the crisis, possibly more people reach dying stage, or people confirmed / number of people infected becomes smaller as there are more peple with it


In [99]:
#@title Death Percentage by filtercoutry
fig = px.line(frame[filterEurope], x='Last Update', y='Death Percentage', color='Country/Region', title='Death Percentage vs Total cases ')
fig.show()

Here I am using filters: 


*   Filter_Date: Gets the lates data for each country
*   Filter_too_little_data: two versions, the first excludes countries with too high death rate, the second countries with less thatn 300 total cases, both of these are outliers that fuck up the stats



In [114]:
#@title Europe death % color map { form-width: "10px" }
#get the last date for which data is available for each country 
frame['Latest update']= frame.groupby([frame['Country/Region']])['Last Update'].transform('max')

filter_Date= frame['Latest update']==frame['Last Update']
filter_too_little_data = ~frame['Country/Region'].isin(['Algeria','Ecuador','Guatemala','Guyana','Sudan', 'Ukraine'])
filter_too_little_data = frame['Total_Cases_by_country']> 300


fig = px.choropleth(data_frame=frame[filter_Europe & filter_Date & filter_too_little_data ], locations='Country/Region', locationmode='country names',  scope="europe", color='Death Percentage', hover_name='Country/Region', hover_data=['Total_deaths_by_country', 'Total_Cases_by_country' ], title='Im Bored at home')
fig.show()



In [115]:
#@title I am filtering out countries with less than 300 cases, cause their stats are retarded
fig = px.choropleth(data_frame=frame[sudan_filter & filter_Date & filter_too_little_data ], locations='Country/Region', locationmode='country names',  scope="world", color='Death Percentage', hover_name='Country/Region', hover_data=['Total_deaths_by_country', 'Total_Cases_by_country' ], title='Im Bored at home')
fig.show()

